In [165]:
import pandas as pd
import numpy as np
import re
import nltk

In [166]:
# Download stopwords dataset
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jacopo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [267]:
df = pd.read_csv('./data/stackechange_csv/beer.stackexchange.com-posts.csv', sep=',')
df.dtypes

Id                 int64
PostTypeId         int64
CreationDate      object
Score              int64
ViewCount        float64
FavoriteCount    float64
Title             object
Body              object
Tags              object
dtype: object

In [268]:
df['Body'].fillna('', inplace=True)
df['Title'].fillna('', inplace=True)
df['Tags'].fillna('', inplace=True)

In [269]:
def remove_html(column):
  df[column] = df[column].str.replace('<.{1,6}>', '')

def apply_lowercase(column):
  df[column] = df[column].str.lower()

def remove_not_alpha(column):
  df[column] = df[column].str.replace('\W', ' ')

def preprocess_body(column):
  remove_html(column)
  apply_lowercase(column)
  remove_not_alpha(column)
  # df[column] = df[column].str.replace('<', '<')\
  #                                         .str.replace('>', '>')\
  #                                         .str.replace('≤', '<=' )\
  #                                         .str.replace('≥', '>=')

  # 2. remove latex i,e., mostly formulas since it's mathematics based dataset
  #df[column] = [re.sub('\$.*?\$', '', i) for i in text]

  # 3. all lowercase 
  # df[column] = df[column].str.lower()

  # 4. decontractions
  # df[column] = df[column].str.replace("won't", "will not").str.replace("can\'t", "can not").str.replace("n\'t", " not").str.replace("\'re", " are").str.\
  #                                               replace("\'s", " is").str.replace("\'d", " would").str.replace("\'ll", " will").str.\
  #                                               replace("\'t", " not").str.replace("\'ve", " have").str.replace("\'m", " am")

  # 5. remove all special-characters other than alpha-numericals
  # df[column] = [re.sub('\W', ' ', i) for i in text]

  # 6. Stop_word removal
  # df[column] = [stopwrd_removal(i) for i in text]

  # 7. remove all white-space i.e., \n, \t, and extra_spaces
  # df[column] = [re.sub('  +', ' ', i) for i in text]
  # df[column] = df[column].str.replace("\n", " ").str.replace("\t", " ").str.strip()


def preprocess_title(column):
  remove_not_alpha(column)
  apply_lowercase(column)

In [270]:
preprocess_title('Title')
preprocess_body('Body')
df

C:\Users\Jacopo\AppData\Local\Temp\ipykernel_7112\3973966932.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('\W', ' ')
C:\Users\Jacopo\AppData\Local\Temp\ipykernel_7112\3973966932.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('<.{1,6}>', '')


,Id,PostTypeId,CreationDate,Score,ViewCount,FavoriteCount,Title,Body,Tags
0,1,1,2014-01-21T20:26:05.383,21,2438.0,NaN,what is a citra hop and how does it differ fr...,i was offered a beer the other day that was re...,<hops>
1,2,1,2014-01-21T20:27:29.797,25,3158.0,NaN,when was the first beer ever brewed,as far as we know when did humans first brew ...,<history>
2,3,1,2014-01-21T20:30:17.437,22,530.0,NaN,how is reduced alcoholic beer made,how is low no alcohol beer made i m assuming ...,<brewing>
3,4,2,2014-01-21T20:31:18.540,15,NaN,NaN,,citra is a registered trademark since 2007 ci...,
4,5,1,2014-01-21T20:31:44.693,22,4028.0,4.0,what temperature should i serve my beer,in general what s the best way to work out th...,<serving><temperature>
...,...,...,...,...,...,...,...,...,...
3813,9500,2,2022-08-02T14:47:40.460,4,NaN,NaN,,is there a standard for spirits and their deca...,
3814,9504,2,2022-08-15T11:56:13.530,0,NaN,NaN,,not an answer on pruning leaves but the first ...,
3815,9508,2,2022-08-23T15:51:15.250,0,NaN,NaN,,one of my old bosses turned me on to jumbo gre...,
3816,9511,2,2022-09-13T20:43:50.437,1,NaN,NaN,,i ve seen bottles of liquor strapped to dogs ...,


In [287]:
# Vocabulary construction
sentences = []
for title in df['Title'].values:
    words = nltk.word_tokenize(title)
    sentences.append(words)

sentences[1]

['when', 'was', 'the', 'first', 'beer', 'ever', 'brewed']

In [288]:
# Remove stopwords from vocabulary
from nltk.corpus import stopwords
# stopwords.words('english')
def remove_stopwords(words):
    return [w for w in words if w not in stopwords.words('english')]

sentences = [remove_stopwords(words) for words in sentences]
    
sentences[1]

['first', 'beer', 'ever', 'brewed']

In [316]:
# Word2Vec model
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

word2vec = Word2Vec(sentences, min_count=10)
word2vec.wv.most_similar('beer')

    

[('name', 0.3637239933013916),
 ('whiskey', 0.26777559518814087),
 ('different', 0.23932719230651855),
 ('breweries', 0.2250223606824875),
 ('drinking', 0.22215844690799713),
 ('ipa', 0.21538133919239044),
 ('spirits', 0.21000239253044128),
 ('cocktail', 0.20445866882801056),
 ('south', 0.19659548997879028),
 ('vodka', 0.19637449085712433)]

In [317]:
# Word2Vec similarity
def word2vec_similarity(model, ws1, ws2):
    """Compute cosine similarity between two sets of words."""
    return model.wv.n_similarity(ws1, ws2)

print(f'Similarity between sentence 1 and 3: ', word2vec_similarity(word2vec, sentences[0], sentences[2]))
print(f'Similarity between sentence 2 and 3: ', word2vec_similarity(word2vec, sentences[1], sentences[2]))


Similarity between sentence 1 and 3:  -0.024783865
Similarity between sentence 2 and 3:  0.5173394


In [174]:
#df['Body']

In [175]:
# TODO: Numerical features encoding